In [ ]:
# module re, datetime, pandas and geopandas
import re, datetime
import pandas as pd
import geopandas as gpd

In [ ]:
# load rawChat data from chat text file
with open("your_whatsapp_chat.txt", "r", encoding="utf8") as f:
    rawChat = f.readlines()

# filter lines with location from chat
chat = []
for line in rawChat:
    if "location: " in line:
        chat.append(line)

In [ ]:
# function for converting chatlines to Final list
chatList = []

for line in chat:
    try:
        #chat line split
        chat_break = re.split(',\ |\ -\ |:\ ', line)
        # join date and time
        separator = " "
        dateTime = separator.join(chat_break[0: 2])
        # convert to date time stamp
        dateTime = datetime.datetime.strptime(dateTime, '%d/%m/%y %I:%M %p')
        # mobile number country code removal and space replace
        number = chat_break[2][4:].replace(" ","")
        # extract coordinates from location URL
        coord = re.split('=', chat_break[-1])[-1][0:-1]
        # split coordinates to lat lng
        lat, lng = float(re.split(',', coord)[0]), float(re.split(',', coord)[1])
        # location URL
        locationURL = chat_break[-1][0:-1]
        # put heading into list for adding to final list
        hList = [dateTime, number, lat, lng, locationURL]
        # appending hList into chatList
        chatList.append(hList)
    except:
        continue

In [ ]:
# chat list to pandas dataframe and geodataframe
columns = ['dateTime', 'mobNumber', 'lat', 'lng', 'locationURL']
chatdf = pd.DataFrame(chatList, columns=columns)
chatgdf = gpd.GeoDataFrame(chatdf, geometry= gpd.points_from_xy(chatdf.lng, chatdf.lat))

In [ ]:
# chat dataframe to csv
chatdf.to_csv("your_whatsapp_chat.csv", index=False)

In [ ]:
#chat dataframe to shp
chatgdf.to_file("your_whatsapp_chat.geojson", driver='GeoJSON')